# Treinamento

## Inicialização

In [1]:
# Config Inicial
import json
import mlflow.pyfunc
import shutil
import pandas as pd
import requests
from sklearn.metrics import r2_score
import mlflow
from mlflow.tracking import MlflowClient

import os
import signal
import subprocess


### Leitura

In [2]:
# Descomentar, quando quiser carregar para o treinamento, dados originais defasados para primeira execução do treinamento
#shutil.copyfile("dataset/brasil_estado_cidade.csv", "dataset/dados_treinamento.csv")

# Carregar o conjunto de dados
df_treinamento = pd.read_csv("dataset/dados_treinamento.csv", encoding="utf-8")

# Eliminando registros com valores null
df_treinamento.dropna(inplace=True)

# float64
df_treinamento = df_treinamento.astype({col: 'float64' for col in df_treinamento.select_dtypes(include='int').columns})

df_treinamento.head()

,id,property_type,state,region,lat,lon,area_m2,price_brl,city
0,1.0,apartment,Pernambuco,Northeast,-8.134204,-34.906326,72.0,414222.98,Recife
1,2.0,apartment,Pernambuco,Northeast,-8.126664,-34.903924,136.0,848408.53,Recife
2,3.0,apartment,Pernambuco,Northeast,-8.125550,-34.907601,75.0,299438.28,Recife
3,4.0,apartment,Pernambuco,Northeast,-8.120249,-34.895920,187.0,848408.53,Recife
4,5.0,apartment,Pernambuco,Northeast,-8.142666,-34.906906,80.0,464129.36,Recife


In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

<Experiment: artifact_location='/app/mlruns/0', creation_time=1744853753613, experiment_id='0', last_update_time=1744853767255, lifecycle_stage='active', name='ecd15', tags={}>

### Amostragem

In [4]:
# amostra
df_sample = df_treinamento.sample(n=20, random_state=42)

x_features = df_sample.drop(["price_brl"], axis=1)  # Features
y_target = df_sample["price_brl"]  # Variável alvo

df_sample.head()

,id,property_type,state,region,lat,lon,area_m2,price_brl,city
3823,3824.0,apartment,Rio de Janeiro,Southeast,-22.930788,-43.355751,62.0,419213.60,Rio de Janeiro
376,377.0,apartment,Piau,Northeast,-5.064457,-42.792750,60.0,319400.84,Teresina
5473,5474.0,apartment,Rio de Janeiro,Southeast,-22.933163,-43.174660,78.0,1287584.69,Rio de Janeiro
2282,2283.0,apartment,Rio Grande do Sul,South,-30.037640,-51.189133,104.0,726908.65,Porto Alegre
980,981.0,apartment,Rio Grande do Sul,South,-30.080242,-51.225227,60.0,494701.71,Porto Alegre


In [5]:
# Gerando o JSON no formato esperado pelo MLflow Serve
dados = {"instances": x_features.to_dict(orient="records")}
json_data = json.dumps(dados)

In [6]:
previsao = requests.post(
    "http://localhost:5000/invocations",
    headers={"Content-Type": "application/json"},
    data=json_data
)
print("HTTP STATUS", previsao.status_code)

HTTP STATUS 200


In [7]:
previsao.text

'{"predictions": [348099.1875, 314764.6875, 963706.625, 730391.0, 330280.34375, 604923.6875, 1018210.0625, 668755.375, 1182253.375, 1222384.375, 798778.8125, 1070625.5, 355035.8125, 863029.625, 542877.5625, 1041795.375, 491103.1875, 373186.9375, 512069.875, 270661.0625]}'

### Carregar e identificar o melhor modelo treinado

In [10]:

def carregar_modelo_com_melhor_r2(experimento_nome):
    client = MlflowClient()

    # Pega o experimento
    experimento = client.get_experiment_by_name(experimento_nome)
    if not experimento:
        raise ValueError(f"Experimento '{experimento_nome}' não encontrado.")
    
    experiment_id = experimento.experiment_id

    melhor_modelo = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    modelo_api = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    # Pega todas as runs ordenadas por R² (decrescente)
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        order_by=["metrics.r2 DESC"]
    )

    modelos_registrados = client.search_registered_models()

    

    for modelo in modelos_registrados:
        
        nome_modelo = modelo.name

        for versao in modelo.latest_versions:
            # Busca informações completas da versão
            versoes = client.search_model_versions(f"name='{nome_modelo}'")
            for v in versoes:
                run = client.get_run(v.run_id)

                # Verifica se pertence ao experimento desejado
                if run.info.experiment_id != experiment_id:
                    continue

                r2 = run.data.metrics.get("r2")

                source_path = v.source
                artifact_path = source_path.split("/")[-1]
                model_uri = f"runs:/{v.run_id}/{artifact_path}"

                print(f"Modelo '{modelo.name}' versão {v.version} da run {v.run_id} com R² = {r2}")

                if v.current_stage == "Production":
                    modelo_api.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
                    continue
                
                if r2 is not None and r2 > melhor_modelo["r2"] and v.current_stage == "Staging":
                    melhor_modelo.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })

    if not melhor_modelo["model_uri"]:
        raise ValueError("Nenhum modelo em 'Staging' com métrica R² encontrado para esse experimento.")

    print(f"Carregando modelo '{melhor_modelo['nome_modelo']}' versão {melhor_modelo['versao_modelo']} da run {melhor_modelo['run_id']} com R² = {melhor_modelo['r2']:.4f}")

    print(f"Modelo API '{modelo_api['nome_modelo']}' versão {modelo_api['versao_modelo']} da run {modelo_api['run_id']} com R² = {modelo_api['r2']:.4f}")
    
    modelo_carregado = mlflow.pyfunc.load_model(melhor_modelo["model_uri"])
    
    return modelo_carregado, melhor_modelo, modelo_api
    

# Carregando modelo com melhor R2 
loaded_model, melhor_modelo, modelo_api = carregar_modelo_com_melhor_r2("ecd15")

melhor_modelo, modelo_api


Modelo 'decision_tree_model' versão 9 da run d60a101ceabd4ca7a3b282552d62d6a3 com R² = 0.44289446310468705
Modelo 'decision_tree_model' versão 8 da run 920600d9a7204c4a8032ca9ad47253bc com R² = 0.4501405374386255
Modelo 'decision_tree_model' versão 7 da run 7dcfc2ff68cd4632aabe1e04c3efc7bf com R² = 0.4662427582353449
Modelo 'decision_tree_model' versão 6 da run cddd012dcec74994979981bcc8eb9d51 com R² = 0.4342869398037643
Modelo 'decision_tree_model' versão 5 da run b42e223e8a8b4127b4dbd355b4834466 com R² = 0.4688968380333659
Modelo 'decision_tree_model' versão 4 da run 26e8268bf94e46c2823560a48c089f9f com R² = 0.4607357460145043
Modelo 'decision_tree_model' versão 3 da run 1bc7cc60346645e9a16531b6064ea027 com R² = 0.47475884579582217
Modelo 'decision_tree_model' versão 2 da run b8798b83b80046d78a75e21ea222c348 com R² = 0.4839914622045445
Modelo 'decision_tree_model' versão 1 da run b1fede94817e4073b2ef3c77bfb7214a com R² = 0.4643648515387715
Modelo 'gradient_boosting_model' versão 9 da

({'r2': 0.7128160768989399,
  'model_uri': 'runs:/e439df4d684e4eae838d4c59cea81ab4/xgboost_model',
  'nome_modelo': 'xgboost_model',
  'versao_modelo': 2,
  'run_id': 'e439df4d684e4eae838d4c59cea81ab4'},
 {'r2': 0.7165948434149507,
  'model_uri': 'runs:/c7f5488202f3431686829d21df15d091/xgboost_model',
  'nome_modelo': 'xgboost_model',
  'versao_modelo': 7,
  'run_id': 'c7f5488202f3431686829d21df15d091'})

## Predição de melhor modelo carregado x modelo API

In [11]:
# Fazer a previsão
predictions = loaded_model.predict(x_features)

# Adicionar as previsões ao DataFrame
df_sample["predicao_melhor_modelo"] = predictions

# converter previsões para o mesmo tipo da variável alvo
df_sample["predicao_melhor_modelo"] = df_sample["predicao_melhor_modelo"].astype(float).round(2)

previsao_json = json.loads(previsao.text)

# Extrair as previsões do JSON e adicioná-las ao DataFrame
df_sample["predicao_modelo_api"] = [round(pred, 2) for pred in previsao_json["predictions"]]

# Exibir o DataFrame atualizado
df_sample[["price_brl", "predicao_modelo_api", "predicao_melhor_modelo"]]

,price_brl,predicao_modelo_api,predicao_melhor_modelo
3823,419213.60,348099.19,319265.22
376,319400.84,314764.69,317740.22
5473,1287584.69,963706.62,983341.94
2282,726908.65,730391.00,696935.06
980,494701.71,330280.34,365088.44
10954,558951.47,604923.69,633396.56
6607,993137.02,1018210.06,1014023.75
1212,678726.79,668755.38,751220.06
6408,1362953.71,1182253.38,1227060.62
11009,1274609.05,1222384.38,1275967.25


### Comparação R² Score dos modelos

In [12]:
# Calcular R²
r2_melhor_modelo = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])
r2_api = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])
print(f"R² API: {r2_melhor_modelo:.4f} R² Melhor Modelo: {r2_api:.4f}")

R² API: 0.8166 R² Melhor Modelo: 0.8166


### Promover melhor modelo para Produção

In [13]:
def parar_modelo_mlflow():
    # Encontra o processo que está usando a porta 5000
    try:
        resultado = subprocess.check_output("lsof -i :5000 -t", shell=True)
        pids = resultado.decode().strip().split("\n")
        for pid in pids:
            os.kill(int(pid), signal.SIGTERM)
        print("Servidor de modelo parado com sucesso.")
    except subprocess.CalledProcessError:
        print("Nenhum processo na porta 5000 foi encontrado.")

def parar_modelo_mlflow1():
    try:
        resultado = subprocess.check_output("lsof -i :5000 -sTCP:LISTEN -nP", shell=True)
        linhas = resultado.decode().strip().split("\n")[1:]  # ignora cabeçalho
        for linha in linhas:
            if "mlflow" in linha.lower():
                pid = int(linha.split()[1])
                os.kill(pid, signal.SIGTERM)
                print(f"Servidor MLflow (PID {pid}) parado com sucesso.")
            else:
                print("⚠️ Porta 5000 ocupada por outro processo que não é o MLflow. Ignorado.")
    except subprocess.CalledProcessError:
        print("Nenhum processo ouvindo na porta 5000 foi encontrado.")

def promover_modelo_para_producao(modelo):
    client = MlflowClient()

    nome_modelo = modelo["nome_modelo"]
    versao_modelo = modelo["versao_modelo"]

    # Promover o modelo selecionado
    client.transition_model_version_stage(
        name=nome_modelo,
        version=versao_modelo,
        stage="Production",
        archive_existing_versions=True  # arquiva automaticamente os outros em Production
    )
    print(f"Modelo '{nome_modelo}' versão {versao_modelo} promovido para 'Production'.")

    # Agora vamos arquivar os que ainda estão em Staging
    todos_modelos = client.search_model_versions("name LIKE '%_model'")
    modelos_ativos = [v for v in todos_modelos if v.current_stage != "Archived"]
    
    for v in modelos_ativos:
        if v.version != versao_modelo and v.name != nome_modelo:
            client.transition_model_version_stage(
                name=nome_modelo,
                version=v.version,
                stage="Archived"
            )
            print(f"Modelo { v.name } Versão {v.version} arquivado.")

def servir_melhor_modelo(experimento_nome, modelo):
    client = MlflowClient()
    
    # Parar modelo atual
    #parar_modelo_mlflow1()

    model_uri = modelo["model_uri"]

    #comando = f"mlflow models serve -m {model_uri} --no-conda --host 0.0.0.0 --port 5000 &"
    #print(f"Executando comando:\n{comando}")
    #get_ipython().system(comando)

    promover_modelo_para_producao(modelo)

servir_melhor_modelo("ecd15", melhor_modelo)

/tmp/ipykernel_236/274293922.py:33: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/tmp/ipykernel_236/274293922.py:47: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Modelo 'xgboost_model' versão 2 promovido para 'Production'.
Modelo gradient_boosting_model Versão 9 arquivado.
Modelo random_forest_model Versão 9 arquivado.
Modelo linear_regression_model Versão 9 arquivado.
Modelo decision_tree_model Versão 9 arquivado.
Modelo gradient_boosting_model Versão 8 arquivado.
Modelo random_forest_model Versão 8 arquivado.
Modelo linear_regression_model Versão 8 arquivado.
Modelo decision_tree_model Versão 8 arquivado.
Modelo gradient_boosting_model Versão 7 arquivado.
Modelo random_forest_model Versão 7 arquivado.
Modelo linear_regression_model Versão 7 arquivado.
Modelo decision_tree_model Versão 7 arquivado.
Modelo gradient_boosting_model Versão 6 arquivado.
Modelo random_forest_model Versão 6 arquivado.
Modelo linear_regression_model Versão 6 arquivado.
Modelo decision_tree_model Versão 6 arquivado.
Modelo gradient_boosting_model Versão 5 arquivado.
Modelo random_forest_model Versão 5 arquivado.
Modelo linear_regression_model Versão 5 arquivado.
Modelo

Modelo decision_tree_model Versão 27 arquivado.


Modelo xgboost_model Versão 26 arquivado.


Modelo gradient_boosting_model Versão 26 arquivado.


...

Modelo decision_tree_model Versão 5 arquivado.
Modelo gradient_boosting_model Versão 4 arquivado.
Modelo linear_regression_model Versão 4 arquivado.


Modelo decision_tree_model Versão 4 arquivado.
Modelo gradient_boosting_model Versão 3 arquivado.
Modelo linear_regression_model Versão 3 arquivado.


Modelo decision_tree_model Versão 3 arquivado.
Modelo gradient_boosting_model Versão 2 arquivado.
Modelo linear_regression_model Versão 2 arquivado.


Modelo decision_tree_model Versão 2 arquivado.
Modelo gradient_boosting_model Versão 1 arquivado.
Modelo linear_regression_model Versão 1 arquivado.


Modelo decision_tree_model Versão 1 arquivado.
